# Using Prototypical Network Model to Build Hierachy map

In [104]:
ls ../data/mini-imagenet/

mini-imagenet-cache-test-complement.pkl
mini-imagenet-cache-test-handout.pkl
mini-imagenet-cache-test.pkl*
mini-imagenet-cache-train-complement.pkl
mini-imagenet-cache-train-handout.pkl
mini-imagenet-cache-train.pkl*
mini-imagenet-cache-val-complement.pkl
mini-imagenet-cache-val-handout.pkl
mini-imagenet-cache-val.pkl*


In [1]:
import os
import json
import math
from tqdm import tqdm
import pickle

import torch
import torchnet as tnt
from torch.autograd import Variable

from visdom import Visdom
from PIL import Image

from functools import reduce
import numpy as np
import copy

viz = Visdom()

## Get handout set

In [107]:
def get_cache_path(split):                                             
    cachePath = ("../data/mini-imagenet/mini-imagenet-cache-" + split + ".pkl")                                                       
    return cachePath  

def savePickle(obj, file):
    with open(file, 'wb') as fh:
        pickle.dump(obj, fh)
        
def split_handout_set(split):
    cachePath = get_cache_path(split)
    with open(cachePath, "rb") as f:                                   
        try:                                                       
            data = pickle.load(f, encoding='bytes')                   
            img_data = data[b'image_data']                         
            class_dict = data[b'class_dict']                       
        except:                                                    
            data = pickle.load(f)                                     
            img_data = data['image_data']                          
            class_dict = data['class_dict']
    handoutSet = {b'image_data': [], b'class_dict': {}}
    handoutSet[b'image_data'] = img_data
    complementarySet = {b'image_data': [], b'class_dict': {}}
    complementarySet[b'image_data'] = img_data
    for k, v in class_dict.items():
        idxs = v
        np.random.shuffle(idxs)
        handoutSet[b'class_dict'][k] = idxs[:200]
        complementarySet[b'class_dict'][k] = idxs[200:]
    savePickle(handoutSet, get_cache_path(split+'-handout'))
    savePickle(complementarySet, get_cache_path(split+'-complement'))
    
splits = ['train', 'val', 'test']
for split in splits:
    split_handout_set(split)



## fine to corase

In [172]:
with open('labelEncoders.pkl', 'rb') as fp:
        labelEncoders = pickle.load(fp)
with open('labelDecoders.pkl', 'rb') as fp:
        labelDecoders = pickle.load(fp)
with open('proto_results/m30_5way5shot/protos.pkl', 'rb') as fp:
        protos_30 = pickle.load(fp)
with open('../labelSemantic.txt', 'r') as fp:       
    wn2name = dict(list(line.strip().split(':')) for line in fp.readlines()) 

In [199]:
import matplotlib.pyplot as plt
import torchvision

for j in range(100):
    ds = []
    for i in range(100):
        ds.append(np.expand_dims(np.power(protos_30[j] - protos_30[i], 2), axis=0))
    ds = np.concatenate(ds, 0)
    ds = ds / np.max(ds) * 255
    imgs = []
    for i in range(64):
        imgs.append(torch.from_numpy(ds[:, i*25:(i+1)*25]).unsqueeze(0))
#     print(torchvision.utils.make_grid(imgs, nrow=64).size())
#     imgs = torch.cat(imgs, 0)
#     print(imgs.unsqueeze(1).size())
    viz.image(torchvision.utils.make_grid(imgs, nrow=64, pad_value=100).numpy(),  
                         opts=dict(title='proto_m30_'+str(j)))
    


In [194]:

for i in range(100):
    print(i, wn2name[labelDecoders[i]])

0.48806792
0 Newfoundland, Newfoundland dog
1 photocopier
2 harvestman, daddy longlegs, Phalangium opilio
3 street sign
4 organ, pipe organ
5 solar dish, solar collector, solar furnace
6 house finch, linnet, Carpodacus mexicanus
7 boxer
8 yawl
9 miniature poodle
10 cliff, drop, drop-off
11 hair slide
12 lipstick, lip rouge
13 jellyfish
14 bolete
15 reel
16 oboe, hautboy, hautbois
17 cocktail shaker
18 barrel, cask
19 Gordon setter
20 fire screen, fireguard
21 Arctic fox, white fox, Alopex lagopus
22 stage
23 tile roof
24 robin, American robin, Turdus migratorius
25 green mamba
26 carousel, carrousel, merry-go-round, roundabout, whirligig
27 holster
28 unicycle, monocycle
29 slot, one-armed bandit
30 worm fence, snake fence, snake-rail fence, Virginia fence
31 dome
32 komondor
33 pencil box, pencil case
34 consomme
35 file, file cabinet, filing cabinet
36 French bulldog
37 orange
38 beer bottle
39 Walker hound, Walker foxhound
40 triceratops
41 dugong, Dugong dugon
42 chime, bell, gong


In [ ]:
n_clusters = 64
from sklearn.cluster import KMeans
from sklearn import metrics

for j in range(100):
    maxs = 0
    maxi = 2
    for i in range(2, n_clusters):
        kmeans = KMeans(n_clusters=i, max_iter=100, random_state=0)
        y_pred = kmeans.fit_predict(probs_features[j]['protos'][1][0].reshape(64, 25))
        score = metrics.calinski_harabaz_score(probs_features[j]['protos'][1][0].reshape(64, 25), y_pred)
        if score > maxs:
            maxi = i
            maxs = score
    print(j, maxi, maxs)

In [239]:
n_clusters = 64
for i in range(2, 10):
    kmeans = KMeans(n_clusters=i, max_iter=300, random_state=0)
    y_pred = kmeans.fit(protos_30[:n_clusters]).predict(protos_30)
    score = metrics.calinski_harabaz_score(protos_30, y_pred)
#         if score > maxs:
#             maxi = i
#             maxs = score
    print(i, score)


2 21.548686322602865
3 19.952476461042547
4 18.277896803927728
5 16.30259049479962
6 14.213483920112878
7 13.965698278219682
8 13.34374567853321
9 13.065374830161902


100


formal: labelSplitProbTable(failed)

LabelEncode | Class 0 prob | Class 1 prob 
----------- | ------------ | ------------ 
     0      |      0.8     |      0.2     
     1      |      0.7     |      0.3     


In [463]:

def getlabelSplitProbTable(protos, centroids):
    def euclidean_dist(x, y):
        # x: N x D
        # y: M x D
        n = x.size(0)
        m = y.size(0)
        d = x.size(1)
        assert d == y.size(1)

        x = x.unsqueeze(1).expand(n, m, d)
        y = y.unsqueeze(0).expand(n, m, d)
        return Variable(torch.pow(x - y, 2).sum(2))
    return torch.nn.Softmax(1)(-euclidean_dist(protos, centroids))
    
label_split_prob_table = getlabelSplitProbTable(
    torch.from_numpy(protos_30), torch.from_numpy(kmeans.cluster_centers_))
label_split_prob_table[0]

Variable containing:
 9.0728e-03
 5.4484e-04
 5.3500e-04
 1.9761e-04
 3.5069e-03
 1.0086e-02
 4.5452e-03
 2.1008e-06
 5.6729e-05
 2.6611e-04
 1.2652e-03
 5.3167e-04
 3.3356e-01
 9.5902e-05
 2.2699e-05
 8.1688e-03
 5.1849e-05
 9.4573e-06
 6.4181e-05
 4.5805e-04
 5.0219e-04
 8.2378e-05
 6.7245e-04
 1.1219e-04
 1.6261e-03
 2.3112e-04
 1.9466e-04
 4.0264e-03
 1.5487e-04
 2.1294e-03
 1.2913e-05
 2.8291e-05
 9.8016e-04
 2.7181e-04
 4.5942e-05
 8.1350e-04
 2.8912e-05
 2.3984e-03
 7.8858e-02
 3.5322e-05
 2.6025e-03
 3.2553e-04
 7.7832e-05
 1.8856e-02
 1.0511e-03
 6.4063e-04
 2.5841e-05
 3.2820e-03
 1.7437e-04
 1.7992e-05
 2.3075e-04
 4.4319e-04
 3.4545e-04
 1.4118e-03
 4.4507e-03
 6.5483e-03
 6.4985e-02
 1.7160e-01
 2.7122e-04
 1.9590e-03
 2.0474e-01
 1.7760e-02
 3.0986e-02
 9.6570e-04
[torch.FloatTensor of size 64]

In [79]:
# import numpy as np
# import pylab as pl

# from sklearn.base import BaseEstimator
# from sklearn.utils import check_random_state
# from sklearn.cluster import MiniBatchKMeans
# from sklearn.cluster import KMeans
# from sklearn.metrics.pairwise import euclidean_distances, manhattan_distances
# from sklearn.datasets.samples_generator import make_blobs

# ##############################################################################
# # Generate sample data
# np.random.seed(0)

# batch_size = 45
# centers = [[1, 1], [-1, -1], [1, -1]]
# n_clusters = len(centers)
# X, labels_true = make_blobs(n_samples=1200, centers=centers, cluster_std=0.3)
# X.shape

In [3]:
with open('HPN_results/m20_5way5shot/protos.pkl', 'rb') as fp:
        probs_features = pickle.load(fp)

In [418]:
from numpy import linalg as LA
dm = []
for i in range(5):
    dm.append([])
    for j in range(5):
        dm[i].append(LA.norm(probs_features[0]['protos'][i] - probs_features[0]['protos'][j], np.inf))


[[0.0, 858.6209, 831.8755, 934.16705, 792.9876],
 [858.6209, 0.0, 845.301, 917.1587, 830.4475],
 [831.8755, 845.301, 0.0, 856.22424, 870.3432],
 [934.16705, 917.1587, 856.22424, 0.0, 859.369],
 [792.9876, 830.4475, 870.3432, 859.369, 0.0]]

# draw data

In [198]:
import matplotlib.pyplot as plt
import torchvision

def calProto(clx):
    clx_s = []
    for i in range(600):
        c_s = []
        for c in range(5):
            c_s.append(probs_features[clx]['protos'][c][i] * probs_features[clx]['prob'][i][c])
        clx_s.append(np.sum(c_s, 0))
    return np.mean(clx_s, 0)

for j in range(100):
    ds = []
    for i in range(100):
        ds.append(np.expand_dims(np.power(calProto(j) - calProto(i), 2), axis=0))
    ds = np.concatenate(ds, 0)
    ds = ds / np.max(ds) * 255
    imgs = []
    for i in range(64):
        imgs.append(torch.from_numpy(ds[:, i*25:(i+1)*25]).unsqueeze(0))
#     print(torchvision.utils.make_grid(imgs, nrow=64).size())
#     imgs = torch.cat(imgs, 0)
#     print(imgs.unsqueeze(1).size())
    viz.image(torchvision.utils.make_grid(imgs, nrow=64, pad_value=100).numpy(),  
                         opts=dict(title='random5_expert_'+str(j)))


# 把类的prototype进行聚类，分析效果

In [256]:
protos_5 = []
# def calProto(clx):
#     clx_s = []
#     for i in range(600):
#         c_s = []
#         for c in range(5):
#             c_s.append(probs_features[clx]['protos'][c][i] * probs_features[clx]['prob'][i][c])
#         clx_s.append(np.sum(c_s, 0))
#     return np.mean(clx_s, 0)
for clx in range(100):
    for i in range(600):
        c_s = []
        for c in range(5):
            c_s.append(probs_features[clx]['protos'][c][i] * probs_features[clx]['prob'][i][c])
        protos_5.append(np.sum(c_s, 0))
print(len(protos_5))

60000


In [261]:
n_clusters = 100
for i in range(2, n_clusters):
    kmeans = KMeans(n_clusters=i, max_iter=300, random_state=0).fit(protos_5)
    y_pred = kmeans.predict(protos_5)
    score = metrics.calinski_harabaz_score(protos_5, y_pred)
#         if score > maxs:
#             maxi = i
#             maxs = score
    print(i, score)
# 2 1648.9918503469153
# 3 1289.362665767065
# 4 1011.7894769748647
# 5 912.7894729263427

2 1672.037734628993


KeyboardInterrupt: 

In [248]:
kmeans.cluster_centers_.shape

(4, 1600)

In [231]:
protos_5c = []
def calProto(clx):
    clx_s = []
    for i in range(600):
        c_s = []
        for c in range(5):
            c_s.append(probs_features[clx]['protos'][c][i] * probs_features[clx]['prob'][i][c])
        clx_s.append(np.sum(c_s, 0))
    return np.mean(clx_s, 0)
for i in range(100):
    protos_5c.append(calProto(i))

# 把整个数据集聚类分析效果

In [266]:
n_clusters = 100
from sklearn.cluster import MiniBatchKMeans, KMeans
for i in range(2, n_clusters+1):
    y_pred = MiniBatchKMeans(n_clusters=i, batch_size = 300, random_state=9).fit(protos_5).predict(protos_5)
    score = metrics.calinski_harabaz_score(protos_5, y_pred)
#         if score > maxs:
#             maxi = i
#             maxs = score
    print(i, score)

2 1665.6801421951639
3 1224.8370157507466
4 1019.2693934122519
5 985.4881128976169
6 834.3001243869725
7 725.9380919378855
8 717.5354767285614
9 666.2287493091318
10 615.2841439729905
11 583.1791682609717
12 522.4971431357803
13 495.36242654497306
14 476.0207804791081
15 449.7065955578855
16 450.1497244891221
17 437.71864094630064
18 386.98879456779673
19 394.9423420838866
20 366.89154131492984
21 362.9014648055912
22 348.94367706137626
23 336.25004670667744
24 333.97464219945795
25 315.2965158345594
26 302.87046912462864
27 299.74467466026283
28 287.56827474202527
29 288.01197584369635
30 280.78147530475934
31 275.62586721064446
32 272.49444528133625
33 252.74448012327395
34 246.48424078820838
35 243.60755750543294
36 237.04188018207995
37 231.5933010963491
38 219.10351858329705
39 222.2356448458348
40 213.84061970708723
41 222.57268413057693
42 208.7858338430881
43 194.86645072961852
44 199.60847925085196
45 196.06197620114665
46 189.19433548350028
47 191.0866527768976
48 185.9245138

In [186]:
import matplotlib.pyplot as plt
import torchvision

for c in range(5):
    for j in range(100):
        ds = []
        for i in range(100):
            ds.append(np.expand_dims(np.power(np.mean(probs_features[i]['protos'][c][:], 0) - 
                               np.mean(probs_features[j]['protos'][c][:], 0), 2), axis=0))
        ds = np.concatenate(ds, 0)
        ds = ds / np.max(ds) * 255
        imgs = []
        for i in range(64):
            imgs.append(torch.from_numpy(ds[:, i*25:(i+1)*25]).unsqueeze(0))
    #     print(torchvision.utils.make_grid(imgs, nrow=64).size())
    #     imgs = torch.cat(imgs, 0)
    #     print(imgs.unsqueeze(1).size())
        viz.image(torchvision.utils.make_grid(imgs, nrow=64, pad_value=100).numpy(),  
                             opts=dict(title='random5_expert_'+str(c)+'_'+str(j)))
    

# 类内距离计算

In [168]:
c = 1

for j in range(10):
    ds = []
    for i in range(100):
        ds.append(np.expand_dims(np.power(probs_features[0]['protos'][c][i] - 
                           probs_features[0]['protos'][c][j], 2), axis=0))
    ds = np.concatenate(ds, 0)
    ds = ds / np.max(ds) * 255
    imgs = []
    for i in range(64):
        imgs.append(torch.from_numpy(ds[:, i*25:(i+1)*25]).unsqueeze(0))
#     print(torchvision.utils.make_grid(imgs, nrow=64).size())
#     imgs = torch.cat(imgs, 0)
#     print(imgs.unsqueeze(1).size())
    viz.image(torchvision.utils.make_grid(imgs, nrow=64, pad_value=100).numpy(),  
                         opts=dict(title=str(j)))
    


(100, 1600)
(100, 1600)
(100, 1600)
(100, 1600)
(100, 1600)
(100, 1600)
(100, 1600)
(100, 1600)
(100, 1600)
(100, 1600)


In [106]:
n_clusters = 64
from sklearn.cluster import KMeans
from sklearn import metrics

for j in range(100):
    maxs = 0
    maxi = 2
    for i in range(2, n_clusters):
        kmeans = KMeans(n_clusters=i, max_iter=100, random_state=0)
        y_pred = kmeans.fit_predict(probs_features[j]['protos'][1][0].reshape(64, 25))
        score = metrics.calinski_harabaz_score(probs_features[j]['protos'][1][0].reshape(64, 25), y_pred)
        if score > maxs:
            maxi = i
            maxs = score
    print(j, maxi, maxs)

0 62 170.15970572311986
1 62 138.37281413475293
2 62 145.6468655257885
3 62 262.76774266685095
4 62 926.1004213096586
5 62 2705.1174530749176
6 62 99.55895968487457
7 62 61.1111944631876
8 62 37.70396970299616
9 62 33.38570216784032
10 62 119.30097323249659
11 62 16.318659634830595
12 62 39.17649542812987
13 62 281.61201444211525
14 62 762.7369848033352
15 62 82.84268017647821
16 62 696.2319390645829
17 62 95.94033229202168
18 62 106.82811976168014
19 62 60.102635930171765
20 61 1512.5104666844306
21 62 60.39055363900232
22 62 350.6330179915364
23 62 172.07847941541576
24 62 42.56976441889585
25 62 4038.01423661421
26 62 584.7371216178752
27 62 44.04059855435547
28 62 69.90206047982404
29 61 7397.401530358475
30 62 54.07379705596237
31 60 12959.158328848986
32 62 69.89393370513145
33 62 50.55182121773055
34 62 64.77680863524894
35 62 312.7674676687571
36 62 46.78089694596426
37 62 671.017355852298
38 62 169.53131114640186
39 62 58.91318360566909
40 62 31.949529378943797
41 58 3286.4435

In [115]:
n_clusters = 64
from sklearn.cluster import KMeans
from sklearn import metrics

for j in range(600):
    i = 62
    kmeans = KMeans(n_clusters=i, max_iter=100, random_state=0)
    y_pred = kmeans.fit_predict(probs_features[0]['protos'][1][j].reshape(64, 25))
    print(".................")
    print([(i, y_pred[i]) for i in range(64)])
    for m in range(62):
        for n in range(m+1, 62):
            if(y_pred[m] == y_pred[n]):
                print(m, n, y_pred[m])
    

.................
[(0, 9), (1, 50), (2, 52), (3, 40), (4, 25), (5, 19), (6, 42), (7, 45), (8, 24), (9, 39), (10, 6), (11, 43), (12, 47), (13, 20), (14, 7), (15, 15), (16, 23), (17, 28), (18, 2), (19, 16), (20, 22), (21, 31), (22, 44), (23, 49), (24, 60), (25, 37), (26, 61), (27, 58), (28, 4), (29, 33), (30, 59), (31, 10), (32, 17), (33, 26), (34, 14), (35, 46), (36, 18), (37, 51), (38, 11), (39, 5), (40, 29), (41, 21), (42, 34), (43, 61), (44, 0), (45, 41), (46, 54), (47, 57), (48, 48), (49, 32), (50, 3), (51, 1), (52, 35), (53, 12), (54, 38), (55, 56), (56, 42), (57, 55), (58, 30), (59, 36), (60, 27), (61, 13), (62, 8), (63, 53)]
6 56 42
26 43 61
.................
[(0, 56), (1, 54), (2, 42), (3, 23), (4, 36), (5, 17), (6, 58), (7, 41), (8, 45), (9, 39), (10, 31), (11, 44), (12, 47), (13, 49), (14, 57), (15, 7), (16, 24), (17, 27), (18, 21), (19, 6), (20, 13), (21, 20), (22, 18), (23, 1), (24, 35), (25, 34), (26, 22), (27, 55), (28, 12), (29, 38), (30, 32), (31, 26), (32, 14), (33, 25)

.................
[(0, 37), (1, 31), (2, 23), (3, 33), (4, 43), (5, 42), (6, 1), (7, 52), (8, 24), (9, 57), (10, 20), (11, 28), (12, 53), (13, 51), (14, 54), (15, 22), (16, 49), (17, 9), (18, 2), (19, 6), (20, 29), (21, 25), (22, 21), (23, 41), (24, 27), (25, 56), (26, 46), (27, 15), (28, 38), (29, 61), (30, 10), (31, 12), (32, 17), (33, 58), (34, 5), (35, 50), (36, 40), (37, 19), (38, 34), (39, 30), (40, 14), (41, 18), (42, 11), (43, 39), (44, 0), (45, 59), (46, 35), (47, 44), (48, 48), (49, 4), (50, 16), (51, 45), (52, 3), (53, 47), (54, 32), (55, 1), (56, 1), (57, 26), (58, 7), (59, 36), (60, 13), (61, 55), (62, 8), (63, 60)]
6 55 1
6 56 1
55 56 1
.................
[(0, 34), (1, 52), (2, 33), (3, 42), (4, 24), (5, 2), (6, 52), (7, 50), (8, 40), (9, 8), (10, 31), (11, 44), (12, 41), (13, 14), (14, 37), (15, 43), (16, 39), (17, 6), (18, 54), (19, 29), (20, 32), (21, 18), (22, 11), (23, 26), (24, 45), (25, 49), (26, 59), (27, 58), (28, 7), (29, 46), (30, 35), (31, 4), (32, 13), (33, 55

.................
[(0, 57), (1, 26), (2, 23), (3, 43), (4, 19), (5, 33), (6, 60), (7, 42), (8, 4), (9, 12), (10, 39), (11, 8), (12, 59), (13, 52), (14, 6), (15, 28), (16, 16), (17, 18), (18, 50), (19, 24), (20, 49), (21, 38), (22, 3), (23, 1), (24, 51), (25, 58), (26, 27), (27, 36), (28, 13), (29, 2), (30, 25), (31, 15), (32, 21), (33, 61), (34, 30), (35, 34), (36, 10), (37, 17), (38, 29), (39, 9), (40, 7), (41, 31), (42, 37), (43, 54), (44, 0), (45, 11), (46, 44), (47, 5), (48, 53), (49, 22), (50, 35), (51, 47), (52, 41), (53, 45), (54, 40), (55, 56), (56, 60), (57, 12), (58, 55), (59, 46), (60, 14), (61, 48), (62, 20), (63, 32)]
6 56 60
9 57 12
.................
[(0, 12), (1, 44), (2, 13), (3, 27), (4, 23), (5, 19), (6, 6), (7, 25), (8, 8), (9, 6), (10, 17), (11, 31), (12, 59), (13, 16), (14, 50), (15, 26), (16, 33), (17, 20), (18, 41), (19, 3), (20, 29), (21, 43), (22, 58), (23, 24), (24, 7), (25, 53), (26, 61), (27, 55), (28, 14), (29, 49), (30, 15), (31, 22), (32, 51), (33, 46), (

.................
[(0, 34), (1, 42), (2, 9), (3, 2), (4, 43), (5, 8), (6, 56), (7, 54), (8, 13), (9, 40), (10, 6), (11, 27), (12, 19), (13, 26), (14, 33), (15, 29), (16, 28), (17, 20), (18, 15), (19, 18), (20, 24), (21, 25), (22, 7), (23, 39), (24, 31), (25, 57), (26, 61), (27, 3), (28, 1), (29, 50), (30, 38), (31, 10), (32, 12), (33, 51), (34, 36), (35, 35), (36, 49), (37, 55), (38, 17), (39, 30), (40, 41), (41, 5), (42, 21), (43, 52), (44, 0), (45, 46), (46, 44), (47, 58), (48, 47), (49, 32), (50, 4), (51, 48), (52, 59), (53, 16), (54, 53), (55, 60), (56, 56), (57, 37), (58, 22), (59, 14), (60, 3), (61, 11), (62, 23), (63, 45)]
6 56 56
27 60 3
.................
[(0, 9), (1, 13), (2, 2), (3, 50), (4, 42), (5, 36), (6, 52), (7, 54), (8, 24), (9, 61), (10, 26), (11, 27), (12, 20), (13, 6), (14, 57), (15, 29), (16, 39), (17, 15), (18, 37), (19, 43), (20, 44), (21, 11), (22, 17), (23, 16), (24, 49), (25, 38), (26, 3), (27, 33), (28, 7), (29, 10), (30, 47), (31, 12), (32, 5), (33, 46), (34

.................
[(0, 31), (1, 52), (2, 42), (3, 9), (4, 27), (5, 8), (6, 32), (7, 56), (8, 29), (9, 4), (10, 6), (11, 24), (12, 51), (13, 28), (14, 36), (15, 23), (16, 43), (17, 15), (18, 45), (19, 17), (20, 44), (21, 41), (22, 3), (23, 25), (24, 50), (25, 49), (26, 53), (27, 57), (28, 13), (29, 61), (30, 54), (31, 7), (32, 10), (33, 48), (34, 34), (35, 32), (36, 14), (37, 20), (38, 21), (39, 12), (40, 37), (41, 16), (42, 33), (43, 22), (44, 0), (45, 60), (46, 38), (47, 5), (48, 40), (49, 26), (50, 2), (51, 1), (52, 47), (53, 18), (54, 30), (55, 59), (56, 32), (57, 35), (58, 19), (59, 11), (60, 55), (61, 46), (62, 39), (63, 58)]
6 35 32
6 56 32
35 56 32
.................
[(0, 16), (1, 42), (2, 15), (3, 23), (4, 24), (5, 54), (6, 53), (7, 40), (8, 24), (9, 56), (10, 6), (11, 36), (12, 26), (13, 29), (14, 43), (15, 7), (16, 31), (17, 20), (18, 18), (19, 13), (20, 39), (21, 28), (22, 10), (23, 21), (24, 37), (25, 41), (26, 27), (27, 51), (28, 34), (29, 49), (30, 25), (31, 2), (32, 4), (

...

.................
[(0, 16), (1, 23), (2, 42), (3, 15), (4, 28), (5, 19), (6, 26), (7, 8), (8, 33), (9, 52), (10, 7), (11, 14), (12, 57), (13, 36), (14, 61), (15, 40), (16, 20), (17, 6), (18, 11), (19, 39), (20, 49), (21, 1), (22, 37), (23, 24), (24, 43), (25, 53), (26, 54), (27, 59), (28, 34), (29, 46), (30, 29), (31, 2), (32, 47), (33, 21), (34, 17), (35, 55), (36, 30), (37, 50), (38, 56), (39, 9), (40, 44), (41, 31), (42, 5), (43, 60), (44, 0), (45, 22), (46, 41), (47, 58), (48, 48), (49, 35), (50, 25), (51, 32), (52, 26), (53, 4), (54, 18), (55, 38), (56, 26), (57, 51), (58, 45), (59, 27), (60, 12), (61, 3), (62, 13), (63, 10)]
6 52 26
6 56 26
52 56 26
.................
[(0, 26), (1, 28), (2, 42), (3, 44), (4, 40), (5, 24), (6, 17), (7, 6), (8, 39), (9, 32), (10, 20), (11, 36), (12, 43), (13, 15), (14, 60), (15, 23), (16, 37), (17, 18), (18, 50), (19, 13), (20, 57), (21, 19), (22, 27), (23, 5), (24, 49), (25, 54), (26, 59), (27, 34), (28, 21), (29, 25), (30, 29), (31, 9), (32, 33), 

.................
[(0, 37), (1, 23), (2, 33), (3, 24), (4, 2), (5, 25), (6, 30), (7, 5), (8, 61), (9, 43), (10, 42), (11, 54), (12, 56), (13, 36), (14, 39), (15, 15), (16, 52), (17, 51), (18, 44), (19, 17), (20, 18), (21, 49), (22, 27), (23, 46), (24, 53), (25, 34), (26, 47), (27, 50), (28, 20), (29, 57), (30, 16), (31, 21), (32, 4), (33, 41), (34, 31), (35, 10), (36, 55), (37, 59), (38, 7), (39, 40), (40, 29), (41, 1), (42, 38), (43, 26), (44, 0), (45, 35), (46, 32), (47, 22), (48, 60), (49, 6), (50, 11), (51, 19), (52, 14), (53, 12), (54, 48), (55, 58), (56, 30), (57, 28), (58, 13), (59, 9), (60, 60), (61, 3), (62, 8), (63, 45)]
6 56 30
48 60 60
.................
[(0, 27), (1, 13), (2, 23), (3, 37), (4, 54), (5, 5), (6, 19), (7, 53), (8, 40), (9, 39), (10, 43), (11, 57), (12, 60), (13, 41), (14, 38), (15, 29), (16, 46), (17, 15), (18, 20), (19, 2), (20, 11), (21, 6), (22, 25), (23, 17), (24, 7), (25, 28), (26, 59), (27, 50), (28, 49), (29, 34), (30, 21), (31, 4), (32, 31), (33, 45), 

.................
[(0, 52), (1, 31), (2, 33), (3, 15), (4, 42), (5, 41), (6, 34), (7, 54), (8, 17), (9, 51), (10, 21), (11, 36), (12, 61), (13, 24), (14, 57), (15, 39), (16, 49), (17, 27), (18, 2), (19, 20), (20, 18), (21, 50), (22, 25), (23, 29), (24, 23), (25, 46), (26, 59), (27, 45), (28, 32), (29, 58), (30, 40), (31, 10), (32, 4), (33, 38), (34, 14), (35, 61), (36, 44), (37, 9), (38, 35), (39, 5), (40, 7), (41, 8), (42, 28), (43, 53), (44, 0), (45, 43), (46, 6), (47, 47), (48, 60), (49, 16), (50, 11), (51, 1), (52, 26), (53, 22), (54, 19), (55, 48), (56, 34), (57, 30), (58, 37), (59, 13), (60, 3), (61, 55), (62, 12), (63, 56)]
6 56 34
12 35 61
.................
[(0, 13), (1, 27), (2, 42), (3, 54), (4, 23), (5, 2), (6, 59), (7, 40), (8, 17), (9, 57), (10, 4), (11, 41), (12, 56), (13, 36), (14, 50), (15, 6), (16, 37), (17, 14), (18, 49), (19, 15), (20, 24), (21, 29), (22, 18), (23, 7), (24, 46), (25, 38), (26, 16), (27, 52), (28, 5), (29, 47), (30, 44), (31, 20), (32, 21), (33, 45), 

.................
[(0, 40), (1, 49), (2, 42), (3, 4), (4, 16), (5, 19), (6, 56), (7, 27), (8, 6), (9, 9), (10, 17), (11, 29), (12, 55), (13, 33), (14, 61), (15, 28), (16, 37), (17, 8), (18, 22), (19, 5), (20, 45), (21, 23), (22, 35), (23, 1), (24, 53), (25, 57), (26, 39), (27, 41), (28, 20), (29, 36), (30, 14), (31, 7), (32, 52), (33, 31), (34, 38), (35, 60), (36, 48), (37, 51), (38, 18), (39, 15), (40, 32), (41, 11), (42, 12), (43, 59), (44, 47), (45, 46), (46, 25), (47, 58), (48, 50), (49, 43), (50, 2), (51, 26), (52, 3), (53, 13), (54, 34), (55, 0), (56, 56), (57, 44), (58, 21), (59, 24), (60, 14), (61, 30), (62, 54), (63, 10)]
6 56 56
30 60 14
.................
[(0, 42), (1, 54), (2, 52), (3, 23), (4, 45), (5, 8), (6, 33), (7, 25), (8, 27), (9, 43), (10, 24), (11, 29), (12, 57), (13, 44), (14, 37), (15, 31), (16, 41), (17, 17), (18, 36), (19, 6), (20, 9), (21, 21), (22, 11), (23, 13), (24, 10), (25, 33), (26, 46), (27, 38), (28, 2), (29, 60), (30, 34), (31, 14), (32, 12), (33, 49),

.................
[(0, 16), (1, 43), (2, 27), (3, 23), (4, 54), (5, 19), (6, 17), (7, 6), (8, 42), (9, 57), (10, 14), (11, 56), (12, 49), (13, 7), (14, 51), (15, 36), (16, 24), (17, 15), (18, 2), (19, 11), (20, 22), (21, 44), (22, 20), (23, 13), (24, 37), (25, 50), (26, 53), (27, 41), (28, 33), (29, 55), (30, 46), (31, 21), (32, 9), (33, 31), (34, 29), (35, 10), (36, 52), (37, 45), (38, 8), (39, 18), (40, 39), (41, 5), (42, 30), (43, 35), (44, 0), (45, 38), (46, 40), (47, 0), (48, 61), (49, 28), (50, 3), (51, 1), (52, 48), (53, 4), (54, 34), (55, 59), (56, 17), (57, 25), (58, 47), (59, 26), (60, 60), (61, 32), (62, 12), (63, 58)]
6 56 17
44 47 0
.................
[(0, 44), (1, 43), (2, 13), (3, 15), (4, 36), (5, 40), (6, 57), (7, 4), (8, 50), (9, 6), (10, 26), (11, 24), (12, 54), (13, 22), (14, 28), (15, 16), (16, 20), (17, 9), (18, 47), (19, 37), (20, 23), (21, 21), (22, 25), (23, 7), (24, 23), (25, 49), (26, 52), (27, 59), (28, 10), (29, 2), (30, 46), (31, 5), (32, 45), (33, 27), (34

In [442]:
probs = [[], [], [], [], []]
print(probs)
probs[fine_to_corase[labelDecoders[64].encode()]].append(probs_features[64]['prob'])
for i in range(64, 100):
#     print(labelDecoders[i], 'idx: ', i, 'belong to: ', fine_to_corase[labelDecoders[i].encode()])
#     print(labelDecoders[i], 'mean prob: ', np.mean(probs_features[i]['prob'], 0))
#     print(labelDecoders[i], 'prob std: ', np.std(probs_features[i]['prob'], 0))
    probs[fine_to_corase[labelDecoders[i].encode()]].append(probs_features[i]['prob'])
for i in range(5):
    print(i)
    np_probs = np.concatenate(probs[i])
    print(np.mean(np_probs, 0))
    print(np.std(np_probs, 0))


[[], [], [], [], []]
0
[0.21758606 0.25140408 0.17391922 0.18138552 0.175705  ]
[0.04839207 0.07378009 0.0355882  0.02612172 0.03964492]
1
[0.19668147 0.2318012  0.18642119 0.18348618 0.20160975]
[0.02971408 0.05773481 0.02986635 0.02077663 0.04178898]
2
[0.2374023  0.22593518 0.17089884 0.18336348 0.18239994]
[0.05450445 0.05013906 0.02441404 0.01889421 0.03378829]
3
[0.1982336  0.2311539  0.18641382 0.18722771 0.19697073]
[0.03232902 0.05470072 0.03078681 0.02302883 0.03926471]
4
[0.2183955  0.24387433 0.17005494 0.19810314 0.16957195]
[0.03571001 0.05017829 0.02792148 0.01887842 0.02964871]


In [395]:
list(range(1,5))

[1, 2, 3, 4]

In [295]:
n = 3
e = torch.exp(torch.DoubleTensor([-300]))
print(e) 
print(torch.log(e))


1.00000e-131 *
  5.1482
[torch.DoubleTensor of size 1]


-300
[torch.DoubleTensor of size 1]



In [543]:
ls ../random*

../random10:
m20_5way5shot/ results/

../random2:
m20_5way5shot/   random2_m20_5_5/

../random3:
m20_5way5shot/

../random32:
m20_5way5shot/

../random32diff:
m20_5way5shot/

../random64:
m20_5way5shot/


In [2]:

import os
import json
import math
from tqdm import tqdm
import pickle

import torch
import torchnet as tnt
from torch.autograd import Variable

from visdom import Visdom
from PIL import Image

viz = Visdom()

from protonets.utils import filter_opt, merge_dict
import protonets.utils.data as data_utils
import protonets.utils.model as model_utils

from functools import reduce
import torch
import torch.nn as nn
import torch.nn.functional as F
model_path = '../random10/m20_5way5shot/best_model.t7'
model = torch.load(model_path)
ps = []
# for i in range(model.n_corase):
#     ps.append(model._parameters['per'+str(i)])
# print(torch.cat(ps, 0))
# print(F.softmax(torch.cat(ps, 0)))
loss_diff = []
mag = []
counter = 0
print(1/model.n_corase)
for i in range(model.n_corase):
    for j in range(i, model.n_corase):
        z1 = model._modules['fine_encoder_'+str(i)][0][0].weight
        z2 = model._modules['fine_encoder_'+str(j)][0][0].weight
        loss_diff.append(torch.pow(z1 - z2, 2))
        counter += 1
for i in range(model.n_corase):
    z = model._modules['fine_encoder_'+str(i)][0][0].weight
    mag.append(torch.pow(z, 2))
print(torch.sum(torch.cat(loss_diff, 0)) / counter)
print(torch.sum(torch.cat(mag, 0)) / model.n_corase)

0.1
Variable containing:
 121.6914
[torch.FloatTensor of size 1]

Variable containing:
 76.4880
[torch.FloatTensor of size 1]



/Users/caixin/anaconda3/lib/python3.6/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class 'protonets.models.few_shot.Protonet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [10]:
a = Variable(torch.Tensor([[1,2,3],[4,5,6]]))
b = a + 1
print(a*b)

Variable containing:
  2   6  12
 20  30  42
[torch.FloatTensor of size 2x3]



In [211]:
0.1789 * 380.5229

68.07554681

In [268]:
from scipy.special import comb, perm
comb(16, 5)

4368.0

In [1]:
import torch
torch.optim

<module 'torch.optim' from '/Users/caixin/anaconda3/lib/python3.6/site-packages/torch/optim/__init__.py'>